<h4>Import Pandas, Beautiful Soup and Requests libraries</h4> 

In [30]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

<h4>Get the requred data and add it to dataframe</h4>
Get the page html from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. Parse it using Beautiful Soup, finding the table with the required data and create the dataframe

In [31]:
page_html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(page_html, 'html.parser')
code_table = soup.find('table', {'class':'wikitable sortable'})
code_table_rows = code_table.find_all('tr')
rows = []
for tr in code_table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        rows.append(row)
df_loc = pd.DataFrame(rows, columns=['PostalCode', 'Borough', 'Neighbourhood'])

<h4>Clean the dataframe</h4>
Add column names then remove all rows where we have <b>Not assigned</b> values in <b>Borough</b> colunm. Replace the <b>Not assigned</b> values in <b>Neighborhood</b> with the corresponding values of <b>Borough</b>.

In [32]:
df_loc = pd.DataFrame(rows, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df_loc = df_loc[df_loc.Borough != 'Not assigned']
#cols = df_loc.columns
#df_loc[cols] = df_loc[cols].mask(df_loc[cols].apply(lambda x: x.str.contains('Not assigned')), df_loc['Borough'], axis=0)

df_loc[['Neighbourhood']] = df_loc[['Neighbourhood']].mask(df_loc[['Neighbourhood']].apply(lambda x: x.str.contains('Not assigned')), df_loc['Borough'], axis=0)

df_loc.index = range(len(df_loc))
df_loc.head(12)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


<h4>Group Boroughs by PostalCode</h4>

In [33]:
df_loc = df_loc.groupby('PostalCode').agg({'Borough':'min', 'Neighbourhood':', '.join})
df_loc.reset_index(inplace=True)
df_loc.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [34]:
df_loc.shape

(103, 3)